## LRGs measuring lines

In [ ]:
# import
import pdb
from astropy import constants as const
import numpy as np
from linetools.isgm.abscomponent import AbsComponent
from linetools import utils as ltu
from pyigm.igm.igmsightline import IGMSightline

import astropy.units as u
from linetools.spectralline import AbsLine, SpectralLine
from linetools import spectralline as ltsp
from linetools.spectra.xspectrum1d import XSpectrum1D
import json

from pyigm.abssys.lls import LLSSystem

In [ ]:
## define:

# jfolder = folder with data

# jsonfs = array with output json files from pyigm_igmguesses

# contfls = array with fits files with continuum spectrum



In [ ]:
def readcomponents(lrgi):
    igm_guess = ltu.loadjson(jfolder+jsonfs[lrgi])
    components = []
    for key in igm_guess['cmps'].keys():
        comp = AbsComponent.from_dict(igm_guess['cmps'][key])#, chk_vel=False)
        components.append(comp)
    igm_sight = IGMSightline.from_components(components, zem=zems[lrgi])
    igm_sys = igm_sight.make_igmsystems()  
    print('igm_sys:')
    for isys in igm_sys:
        print(isys)
    return igm_sys

def writecomp(ii,lrgi,igm_sys,namecf=''):
    print(igm_sys[ii].zabs)
    for icomp in igm_sys[ii]._components:
            print(icomp)
    lrg = igm_sys[ii]
    print(lrg.vlim)
    if namecf == '':
        lrg.write_json(jfolder+jsonfs[lrgi][0:15]+'LRG_'+jsonfs[lrgi][0:5]+'.json')
    else:
        lrg.write_json(jfolder+jsonfs[lrgi][0:15]+namecf)


In [ ]:
lrgi = 0
igm_sys = readcomponents(lrgi)
ii = 0 # choose the #component which corresponds to LRG
writecomp(ii,lrgi,igm_sys)

### the same for lrgi = 0 - 14

In [ ]:
## copy in terminal
for i in np.arange(len(contfls)):
    print('lt_xabssys '+contfls[i]+' '+jfolder+jsonfs[i][0:15]+'LRG_'+jsonfs[i][0:5]+'.json'+' -o '+jsonfs[i][0:5]+'_xabssys.json')
    print(' ')


In [ ]:
## print EWs

#all_ew={}

for i in np.arange(15):
    print(xabssfiles[i])
    lls = LLSSystem.from_json(xabssfiles[i], chk_vel=False)
    spec_file = jfolder + contfls[i]
    abs_lines = lls.list_of_abslines()
    #all_ew[str(i)]={}
    
    for absline in abs_lines:
        # Spectrum
        #if spec is None:
        #    spec_file = grab_specfile(absline)
            # Read
        spec = lsio.readspec(spec_file)
        #else:  # Check it is the same one
        #    new_spec_file = grab_specfile(absline)
        #    if new_spec_file != spec_file:
        #        raise ValueError("Not ready for this..")
        absline.analy['spec'] = spec
        # EW
        absline.measure_restew(flg=1)  # Boxcar
        # AODM
        absline.measure_aodm()

        print(absline)
        print('EW = {:g} with error {:g}'.format(absline.attrib['EW'],absline.attrib['sig_EW']))
        print(' ')
        #all_ew[str(i)][absline]=absline.attrib['EW']
    print('====================================')
    print('   ')
